In [106]:
#parameters
n = 6
m = 19
t = 9
s1 = 'RELIANCE.NS'
s2 = 'JSWSTEEL.NS'

In [107]:
import numpy as np
import pandas as pd

In [108]:
#function to calculate macd
def MACD(data, short_window, long_window, signal_window, column_name):
   
    data.loc[:,'EMAn'] = data[column_name].ewm(span=short_window, adjust=False).mean()
    data.loc[:,'EMAm'] = data[column_name].ewm(span=long_window, adjust=False).mean()

    for i in range(short_window):
        data.loc[i,'EMAn'] = None
    for i in range(long_window):
        data.loc[i,'EMAm'] = None
    
    data['MACD'] = data['EMAn'] - data['EMAm']
    data['Signal Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()

    trade_sig=np.zeros([len(data),1],dtype=float)

    for i in range(long_window+1,len(data)):
        if data.loc[i,'MACD'] > data.loc[i,'Signal Line'] and data.loc[i-1,'MACD'] < data.loc[i-1,'Signal Line']:
            trade_sig[i] = 1
        if data.loc[i,'MACD'] < data.loc[i,'Signal Line'] and data.loc[i-1,'MACD'] > data.loc[i-1,'Signal Line']:
            trade_sig[i] = -1
        
    trade_sig=trade_sig.reshape(len(trade_sig),1)

    data_numpy=data['Date'].to_numpy()
    data_numpy=data_numpy.reshape(len(data_numpy),1)
    data_numpy=np.column_stack((data_numpy,data[column_name].to_numpy()))
    data_numpy=np.column_stack((data_numpy,data['EMAn'].to_numpy()))
    data_numpy=np.column_stack((data_numpy,data['EMAm'].to_numpy()))
    data_numpy=np.column_stack((data_numpy,data['MACD'].to_numpy()))
    data_numpy=np.column_stack((data_numpy,data['Signal Line'].to_numpy()))

    final=np.column_stack((data_numpy,trade_sig))
    final_df=pd.DataFrame(final,columns=['Date','Close','EMAn','EMAm','MACD','Signal Line','Trade Signal'],index=data.index)

    final_df.drop(["EMAn","EMAm","MACD","Signal Line"],axis=1,inplace=True)
    final_df['Date'] = pd.to_datetime(final_df['Date'])
    final_df['Close'] = final_df['Close'].astype(float) 
    final_df['Trade Signal'] = final_df['Trade Signal'].astype(int)

    return final_df

In [109]:
data = pd.read_csv('/Users/hitengoel/Machine Learning/data analysis(technical indiactors)/data-analysis-with-technical-indicators/Pair trading/Returns data/sensex_close.csv')


In [110]:
def get_master_macd(x,y,s1,s2) :
    master_macd = pd.DataFrame()
    master_macd['Date'] = x['Date']
    master_macd[s1] = x['Trade Signal']
    master_macd[s2] = y['Trade Signal']
    return master_macd


In [111]:
def get_profit(orders):
    profit = 0
    for i in range(len(orders)):
        if(orders.iloc[i]['Order'] == 'Buy'):
            profit -= orders.iloc[i]['Price']
        else:
            profit += orders.iloc[i]['Price']
    return profit

In [112]:
prices = pd.read_csv('/Users/hitengoel/Machine Learning/data analysis(technical indiactors)/data-analysis-with-technical-indicators/Pair trading/Returns data/sensex_close.csv')

for i in range(1,len(prices.columns)):
    prices.columns.values[i] = prices.columns.values[i][:-7]

In [113]:
x = MACD(data, n, m, t, s1+'(close)')
y = MACD(data, n, m, t, s2+'(close)')
master_macd = get_master_macd(x,y,s1,s2)
groups = [[s1,s2]]

prices = pd.read_csv('/Users/hitengoel/Machine Learning/data analysis(technical indiactors)/data-analysis-with-technical-indicators/Pair trading/Returns data/sensex_close.csv')

for i in range(1,len(prices.columns)):
    prices.columns.values[i] = prices.columns.values[i][:-7]


In [114]:
intrade = []
orders = pd.DataFrame(columns=['Date', 'Stock', 'Order', 'Price'])

for k in groups:
    for i in range(len(master_macd)):
        row = master_macd.iloc[i]
        buys = 0
        sells = 0
        for j in range(1,len(row)):
            if((master_macd.columns[j] in k) and master_macd.iloc[i,j] == 1):
                buys += 1
            elif((master_macd.columns[j] in k) and master_macd.iloc[i,j] == -1):
                sells += 1
        if(buys < 1 or sells < 1):
            continue
        
        count = min(buys,sells)
        buy_stocks = []
        sell_stocks = []

        for j in range(1,len(row)):
            if((master_macd.columns[j] in k) and master_macd.iloc[i,j] == 1 and len(buy_stocks) < count):
                buy_stocks.append(master_macd.columns[j])
            elif((master_macd.columns[j] in k) and master_macd.iloc[i,j] == -1 and len(sell_stocks) < count):
                sell_stocks.append(master_macd.columns[j])
        

        for l in range(len(buy_stocks)):
            for m in range(len(sell_stocks)):
                if(buy_stocks[l] == '' or sell_stocks[m] == ''):
                    continue
                if([buy_stocks[l], sell_stocks[m]] in intrade):
                    continue
                
                #for q in intrade:
                 #   if(buy_stocks[l] in q or sell_stocks[m] in q):
                  #      orders = pd.concat([orders,pd.DataFrame({'Date': [master_macd.iloc[i]['Date']], 'Stock': [q[0]], 'Order': ['Sell'], 'Price': [prices.iloc[i][q[0]]]})], ignore_index=True)
                   #     orders = pd.concat([orders,pd.DataFrame({'Date': [master_macd.iloc[i]['Date']], 'Stock': [q[1]], 'Order': ['Buy'], 'Price': [prices.iloc[i][q[1]]]})], ignore_index=True)
                    #    intrade.remove(q)
                
                intrade.append([buy_stocks[l], sell_stocks[m]])
                orders = pd.concat([orders,pd.DataFrame({'Date': [master_macd.iloc[i]['Date']], 'Stock': [buy_stocks[l]], 'Order': ['Buy'], 'Price': [prices.iloc[i][buy_stocks[l]]]})], ignore_index=True)
                orders = pd.concat([orders,pd.DataFrame({'Date': [master_macd.iloc[i]['Date']], 'Stock': [sell_stocks[m]], 'Order': ['Sell'], 'Price': [prices.iloc[i][sell_stocks[m]]]})], ignore_index=True)

                buy_stocks[l] = ''
                sell_stocks[m] = ''


    while(len(intrade) > 0):
        q = intrade.pop()
        orders = pd.concat([orders,pd.DataFrame({'Date': [master_macd.iloc[-1]['Date']], 'Stock': [q[0]], 'Order': ['Sell'], 'Price': [prices.iloc[-1][q[0]]]})], ignore_index=True)
        orders = pd.concat([orders,pd.DataFrame({'Date': [master_macd.iloc[-1]['Date']], 'Stock': [q[1]], 'Order': ['Buy'], 'Price': [prices.iloc[-1][q[1]]]})], ignore_index=True)


/var/folders/94/qzww3__j7w95pdc_5q59f29c0000gn/T/ipykernel_46560/3019151391.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  orders = pd.concat([orders,pd.DataFrame({'Date': [master_macd.iloc[i]['Date']], 'Stock': [buy_stocks[l]], 'Order': ['Buy'], 'Price': [prices.iloc[i][buy_stocks[l]]]})], ignore_index=True)


In [115]:
profit = get_profit(orders)
profit

122.59892272949219